In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_Kmer"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

# monitor = 'val_loss'

In [2]:
train_data_filename = 'Training-datasets-PredNTS_kmer.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kmer.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

import math

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [6]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [7]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [8]:
# monitor = 'val_loss'

# epochs = 100
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 2048, ## Dense layer parameters,
#                     dense_layers = 5,
#                     prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)

#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [9]:
monitor = 'val_accuracy'

epochs = 100
batch_size = 32

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 2048, ## Dense layer parameters,
                    dense_layers = 5,
                    prob = 0.5, learn_rate = 0.0005, 
                    gn = 0.01,
                    loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
    y = tf.keras.layers.GaussianNoise(stddev=gn)(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    for i in range(1,dense_layers+1):
    
        y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
                                  kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                 )(y)
        y = tf.keras.layers.GaussianNoise(stddev=gn)(y)
        y = tf.keras.layers.Dropout(prob)(y) 
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [10]:
# epochs = 100
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [11]:
# epochs = 200
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 16, ## Dense layer parameters,
#                     dense_layers = 3,
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [12]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob_input = 0.9, prob = 0.5,
#                     learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dropout(prob_input)(input1)
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(y)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [13]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [14]:
DLNN_Classifier((8420)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8420)]            0         
                                                                 
 dense (Dense)               (None, 2048)              17246208  
                                                                 
 gaussian_noise (GaussianNoi  (None, 2048)             0         
 se)                                                             
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 gaussian_noise_1 (GaussianN  (None, 1024)             0         
 oise)                                                       

# Train data preparation

In [15]:
##################################################################################
##### Read CSV data
##################################################################################

train_data_filepath = os.path.join(input_data_folder, train_data_filename)
train_data = pd.read_csv(train_data_filepath, sep=',', header=0)
train_data = train_data.drop('label', axis=1)

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

input_vec_shape = train_features[0].shape

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Training

In [16]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/200
60/60 [==============================] - ETA: 0s - loss: 4.4784 - accuracy: 0.5370
Epoch 1: val_accuracy improved from -inf to 0.76520, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kmer\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 3s 30ms/step - loss: 4.4784 - accuracy: 0.5370 - val_loss: 3.4754 - val_accuracy: 0.7652
Epoch 2/200
56/60 [===========================>..] - ETA: 0s - loss: 3.1859 - accuracy: 0.7087
Epoch 2: val_accuracy improved from 0.76520 to 0.77568, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kmer\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 1s 24ms/step - loss: 3.1726 - accuracy: 0.7102 - val_loss: 2.9234 - val_accuracy: 0.7757
Epoch 3/200
56/60 [===========================>..] - ETA: 0s - loss: 2.6982 - accuracy: 0.8259
Epoch 3: val_accuracy did not improve from 0.77568
60/60 [==============================] - 1s 13ms/step - loss: 2.688

Epoch 29/200
56/60 [===========================>..] - ETA: 0s - loss: 0.5544 - accuracy: 0.9570
Epoch 29: val_accuracy did not improve from 0.77987
60/60 [==============================] - 1s 13ms/step - loss: 0.5542 - accuracy: 0.9575 - val_loss: 1.3074 - val_accuracy: 0.7526
Epoch 30/200
56/60 [===========================>..] - ETA: 0s - loss: 0.4912 - accuracy: 0.9749
Epoch 30: val_accuracy did not improve from 0.77987
60/60 [==============================] - 1s 12ms/step - loss: 0.4915 - accuracy: 0.9738 - val_loss: 1.3581 - val_accuracy: 0.7715
Epoch 31/200
60/60 [==============================] - ETA: 0s - loss: 0.4833 - accuracy: 0.9685
Epoch 31: val_accuracy did not improve from 0.77987
60/60 [==============================] - 1s 13ms/step - loss: 0.4833 - accuracy: 0.9685 - val_loss: 1.3854 - val_accuracy: 0.7715
Epoch 32/200
56/60 [===========================>..] - ETA: 0s - loss: 0.5196 - accuracy: 0.9542
Epoch 32: val_accuracy did not improve from 0.77987
60/60 [===========

56/60 [===========================>..] - ETA: 0s - loss: 0.3966 - accuracy: 0.9732
Epoch 58: val_accuracy did not improve from 0.78616
60/60 [==============================] - 1s 13ms/step - loss: 0.3958 - accuracy: 0.9722 - val_loss: 1.3053 - val_accuracy: 0.7589
Epoch 59/200
56/60 [===========================>..] - ETA: 0s - loss: 0.4036 - accuracy: 0.9682
Epoch 59: val_accuracy did not improve from 0.78616
60/60 [==============================] - 1s 13ms/step - loss: 0.4016 - accuracy: 0.9685 - val_loss: 1.3293 - val_accuracy: 0.7652
Epoch 60/200
56/60 [===========================>..] - ETA: 0s - loss: 0.4159 - accuracy: 0.9626
Epoch 60: val_accuracy did not improve from 0.78616
60/60 [==============================] - 1s 13ms/step - loss: 0.4140 - accuracy: 0.9633 - val_loss: 1.5373 - val_accuracy: 0.7484
Epoch 61/200
56/60 [===========================>..] - ETA: 0s - loss: 0.4135 - accuracy: 0.9704
Epoch 61: val_accuracy did not improve from 0.78616
60/60 [========================

60/60 [==============================] - ETA: 0s - loss: 0.4559 - accuracy: 0.9538
Epoch 87: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.4559 - accuracy: 0.9538 - val_loss: 1.2657 - val_accuracy: 0.7505
Epoch 88/200
56/60 [===========================>..] - ETA: 0s - loss: 0.3942 - accuracy: 0.9749
Epoch 88: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.3913 - accuracy: 0.9759 - val_loss: 1.5256 - val_accuracy: 0.7484
Epoch 89/200
60/60 [==============================] - ETA: 0s - loss: 0.3860 - accuracy: 0.9743
Epoch 89: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.3860 - accuracy: 0.9743 - val_loss: 1.3346 - val_accuracy: 0.7673
Epoch 90/200
60/60 [==============================] - ETA: 0s - loss: 0.3836 - accuracy: 0.9722
Epoch 90: val_accuracy did not improve from 0.79036
60/60 [========================

60/60 [==============================] - 1s 13ms/step - loss: 0.3710 - accuracy: 0.9785 - val_loss: 1.4805 - val_accuracy: 0.7652
Epoch 117/200
60/60 [==============================] - ETA: 0s - loss: 0.3894 - accuracy: 0.9732
Epoch 117: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.3894 - accuracy: 0.9732 - val_loss: 1.3489 - val_accuracy: 0.7673
Epoch 118/200
56/60 [===========================>..] - ETA: 0s - loss: 0.3962 - accuracy: 0.9710
Epoch 118: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.3946 - accuracy: 0.9717 - val_loss: 1.4121 - val_accuracy: 0.7631
Epoch 119/200
60/60 [==============================] - ETA: 0s - loss: 0.3501 - accuracy: 0.9764
Epoch 119: val_accuracy did not improve from 0.79036
60/60 [==============================] - 1s 13ms/step - loss: 0.3501 - accuracy: 0.9764 - val_loss: 1.5415 - val_accuracy: 0.7547
Epoch 120/200
58/60 [=========

...

60/60 [==============================] - 1s 13ms/step - loss: 0.4229 - accuracy: 0.9717 - val_loss: 1.0174 - val_accuracy: 0.7962
Epoch 71/200
58/60 [============================>.] - ETA: 0s - loss: 0.4142 - accuracy: 0.9774
Epoch 71: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.4177 - accuracy: 0.9774 - val_loss: 1.0673 - val_accuracy: 0.7983
Epoch 72/200
60/60 [==============================] - ETA: 0s - loss: 0.4313 - accuracy: 0.9622
Epoch 72: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.4313 - accuracy: 0.9622 - val_loss: 0.9272 - val_accuracy: 0.8046
Epoch 73/200
59/60 [============================>.] - ETA: 0s - loss: 0.4414 - accuracy: 0.9645
Epoch 73: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.4409 - accuracy: 0.9648 - val_loss: 1.1130 - val_accuracy: 0.7920
Epoch 74/200
59/60 [================

58/60 [============================>.] - ETA: 0s - loss: 0.4054 - accuracy: 0.9747
Epoch 100: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.4058 - accuracy: 0.9743 - val_loss: 1.0786 - val_accuracy: 0.8004
Epoch 101/200
56/60 [===========================>..] - ETA: 0s - loss: 0.4075 - accuracy: 0.9682
Epoch 101: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.4041 - accuracy: 0.9696 - val_loss: 1.0769 - val_accuracy: 0.8025
Epoch 102/200
60/60 [==============================] - ETA: 0s - loss: 0.3942 - accuracy: 0.9685
Epoch 102: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3942 - accuracy: 0.9685 - val_loss: 0.9936 - val_accuracy: 0.7941
Epoch 103/200
59/60 [============================>.] - ETA: 0s - loss: 0.3858 - accuracy: 0.9735
Epoch 103: val_accuracy did not improve from 0.82143
60/60 [=================

59/60 [============================>.] - ETA: 0s - loss: 0.3799 - accuracy: 0.9746
Epoch 129: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3807 - accuracy: 0.9743 - val_loss: 1.0617 - val_accuracy: 0.7920
Epoch 130/200
56/60 [===========================>..] - ETA: 0s - loss: 0.3726 - accuracy: 0.9766
Epoch 130: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 14ms/step - loss: 0.3739 - accuracy: 0.9759 - val_loss: 1.0468 - val_accuracy: 0.8004
Epoch 131/200
60/60 [==============================] - ETA: 0s - loss: 0.3857 - accuracy: 0.9717
Epoch 131: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3857 - accuracy: 0.9717 - val_loss: 1.0926 - val_accuracy: 0.7962
Epoch 132/200
59/60 [============================>.] - ETA: 0s - loss: 0.4364 - accuracy: 0.9592
Epoch 132: val_accuracy did not improve from 0.82143
60/60 [=================

58/60 [============================>.] - ETA: 0s - loss: 0.3633 - accuracy: 0.9747
Epoch 158: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3633 - accuracy: 0.9743 - val_loss: 1.0418 - val_accuracy: 0.8004
Epoch 159/200
60/60 [==============================] - ETA: 0s - loss: 0.3521 - accuracy: 0.9785
Epoch 159: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3521 - accuracy: 0.9785 - val_loss: 1.1563 - val_accuracy: 0.7941
Epoch 160/200
59/60 [============================>.] - ETA: 0s - loss: 0.3424 - accuracy: 0.9815
Epoch 160: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3421 - accuracy: 0.9816 - val_loss: 1.1212 - val_accuracy: 0.8004
Epoch 161/200
59/60 [============================>.] - ETA: 0s - loss: 0.3831 - accuracy: 0.9746
Epoch 161: val_accuracy did not improve from 0.82143
60/60 [=================

56/60 [===========================>..] - ETA: 0s - loss: 0.3521 - accuracy: 0.9766
Epoch 187: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3509 - accuracy: 0.9769 - val_loss: 1.0162 - val_accuracy: 0.8004
Epoch 188/200
56/60 [===========================>..] - ETA: 0s - loss: 0.3574 - accuracy: 0.9715
Epoch 188: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3665 - accuracy: 0.9680 - val_loss: 0.9371 - val_accuracy: 0.8025
Epoch 189/200
60/60 [==============================] - ETA: 0s - loss: 0.3628 - accuracy: 0.9727
Epoch 189: val_accuracy did not improve from 0.82143
60/60 [==============================] - 1s 13ms/step - loss: 0.3628 - accuracy: 0.9727 - val_loss: 1.1486 - val_accuracy: 0.7962
Epoch 190/200
58/60 [============================>.] - ETA: 0s - loss: 0.3490 - accuracy: 0.9768
Epoch 190: val_accuracy did not improve from 0.82143
60/60 [=================

## k-fold Training evaluation

In [17]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.800166,0.810883,0.859428,0.787588,0.812689,0.602598
Train,0.987931,0.985718,0.998322,0.990345,0.985518,0.975941


In [18]:
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.804782	0.822802	0.861915	0.777494	0.832056	0.610976
# Train	0.991184	0.993261	0.997514	0.989086	0.993283	0.982392

In [19]:
evaluations_df.groupby(["Train_Test"]).std().filter(['Accuracy', 
                                                       'Precision', 
                                                       'AUC', 
                                                       'Sensitivity', 
                                                       'Specificity', 
                                                       'MCC'])

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.018178,0.042840,0.022797,0.041340,0.055795,0.038056
Train,0.007456,0.012992,0.000739,0.004536,0.013383,0.014792


In [20]:
# small 8,2 - 200,16
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.798895	0.802586	0.872018	0.792578	0.805204	0.597894
# Train	0.995382	0.996426	0.999765	0.994333	0.996431	0.990768

In [21]:
# small 16, 3
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.787978	0.796841	0.865977	0.773267	0.802676	0.576380
# Train	0.995697	0.996009	0.999902	0.995382	0.996012	0.991395

In [22]:
# small 8, 2
# Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.795976	0.807037	0.866398	0.778376	0.813600	0.592587
# Train	0.995697	0.996638	0.999890	0.994752	0.996642	0.991400

In [23]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.995276,0.996839,"[0.0, 0.0021008403361344537, 0.004201680672268...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999999, 0.9999999, 0.99999976, 0.9999993, ...",0.998789,0.993697,0.996852,0.990556
1,0,Test,0.790356,0.803493,"[0.0, 0.0041841004184100415, 0.012552301255230...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999998, 0.99999976, 0.9999958, 0.99999285,...",0.834289,0.769874,0.810924,0.581259
2,1,Train,0.987402,0.991534,"[0.0, 0.001049317943336831, 0.0031479538300104...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.9999994, 0.99999833, 0...",0.997115,0.983211,0.991597,0.974838
3,1,Test,0.815514,0.878788,"[0.0, 0.004201680672268907, 0.0714285714285714...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[2.0, 1.0, 0.99970067, 0.99963355, 0.99957556,...",0.881254,0.731092,0.899582,0.639967
4,2,Train,0.992655,0.992655,"[0.0, 0.00944386149003148, 0.01678908709338929...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.998566,0.992655,0.992655,0.985310
5,2,Test,0.796218,0.802575,"[0.0, 0.008403361344537815, 0.0252100840336134...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[2.0, 1.0, 0.99999976, 0.9999995, 0.9999554, 0...",0.847821,0.785714,0.806723,0.592568
6,3,Train,0.975866,0.964176,"[0.0, 0.001049317943336831, 0.0115424973767051...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.99999344, 0.9999927, 0.9999895, 0...",0.998164,0.988458,0.963274,0.952033
7,3,Test,0.777311,0.759843,"[0.0, 0.004201680672268907, 0.1386554621848739...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999993, 0.9999993, 0.9985587, 0.998487, 0....",0.847857,0.810924,0.743697,0.555879
8,4,Train,0.988458,0.983385,"[0.0, 0.001049317943336831, 0.0031479538300104...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.99999976, 0.9999993, 0.99999917, ...",0.998975,0.993704,0.983211,0.976969
9,4,Test,0.821429,0.809717,"[0.0, 0.004201680672268907, 0.1176470588235294...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999987, 0.9999987, 0.9996489, 0.9996474, 0...",0.885919,0.840336,0.802521,0.643317


In [24]:
# small 8, 2
# 	Fold	Train_Test	Accuracy	Precision	TPR	FPR	TPR_FPR_Thresholds	AUC	Sensitivity	Specificity	MCC
# 0	0	Train	0.994751	0.994748	[0.0, 0.0010504201680672268, 0.103991596638655...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[1.9849311, 0.9849311, 0.92211884, 0.92180663,...	0.999930	0.994748	0.994753	0.989501
# 1	0	Test	0.771488	0.795455	[0.0, 0.0041841004184100415, 0.020920502092050...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9901093, 0.9901093, 0.9751286, 0.9726768, 0...	0.857600	0.732218	0.810924	0.544782
# 2	1	Train	0.994751	0.997888	[0.0, 0.001049317943336831, 0.1248688352570829...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[1.9553037, 0.9553036, 0.89843416, 0.89823747,...	0.999837	0.991605	0.997899	0.989521
# 3	1	Test	0.805031	0.811159	[0.0, 0.004201680672268907, 0.0462184873949579...	[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...	[1.9966587, 0.9966587, 0.96096516, 0.96064854,...	0.885157	0.794118	0.815900	0.610178
# 4	2	Train	0.997901	0.996859	[0.0, 0.001049317943336831, 0.1101783840503672...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[1.9665295, 0.9665295, 0.92234534, 0.9217606, ...	0.999968	0.998951	0.996852	0.995805
# 5	2	Test	0.779412	0.775934	[0.0, 0.004201680672268907, 0.0546218487394958...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9883118, 0.9883118, 0.957935, 0.9533922, 0....	0.838580	0.785714	0.773109	0.558868
# 6	3	Train	0.995803	0.995803	[0.0, 0.001049317943336831, 0.1185729275970619...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[1.9773536, 0.9773536, 0.93449837, 0.9342757, ...	0.999827	0.995803	0.995803	0.991605
# 7	3	Test	0.836134	0.850877	[0.0, 0.004201680672268907, 0.0378151260504201...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.986332, 0.98633206, 0.97150844, 0.97079784,...	0.889556	0.815126	0.857143	0.672863
# 8	4	Train	0.995278	0.997890	[0.0, 0.001049317943336831, 0.1049317943336831...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[1.961971, 0.961971, 0.9118028, 0.9117352, 0.9...	0.999890	0.992655	0.997901	0.990570
# 9	4	Test	0.787815	0.801762	[0.0, 0.004201680672268907, 0.0504201680672268...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9933276, 0.9933276, 0.9593585, 0.95925444, ...	0.861097	0.764706	0.810924	0.576246

# Independent data

In [25]:
##################################################################################
##### Read CSV data
##################################################################################

indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename)
indpe_data = pd.read_csv(indpe_data_filepath, sep=',', header=0)
indpe_data = indpe_data.drop('label', axis=1)

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

## Using k-fold Models

### Performance of each k-fold model

In [26]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.586939,0.225012,0.638483,0.609852,0.582387,0.144085


In [27]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.595102,0.223062,"[0.0, 0.0049261083743842365, 0.019704433497536...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9999998, 0.99999976, 0.9999993, 0.9999963, ...",0.640777,0.581281,0.597847,0.134464
1,1,Independent,0.619592,0.225470,"[0.0, 0.009852216748768473, 0.0147783251231527...","[0.0, 0.0, 0.0, 0.0029354207436399216, 0.00293...","[1.9999999, 0.9999999, 0.99999976, 0.9999982, ...",0.621634,0.532020,0.636986,0.128777
2,2,Independent,0.600816,0.233209,"[0.0, 0.019704433497536946, 0.0246305418719211...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[2.0, 1.0, 0.9999999, 0.99999976, 0.9999995, 0...",0.637189,0.615764,0.597847,0.160105
3,3,Independent,0.563265,0.226073,"[0.0, 0.0049261083743842365, 0.024630541871921...","[0.0, 0.0, 0.0, 0.004892367906066536, 0.006849...","[1.9999994, 0.9999994, 0.99999154, 0.99998355,...",0.654960,0.674877,0.541096,0.160617
4,4,Independent,0.555918,0.217247,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0, 0.0009784735812133072, 0....","[1.9999996, 0.99999964, 0.99999917, 0.9999989,...",0.637854,0.645320,0.538160,0.136461


### Mean score with k-fold models

In [28]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.601633,0.231638,0.647562,0.605911,0.600783,0.155087


### Voting score with k-fold models

In [29]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.596735,0.231054,0.625382,0.615764,0.592955,0.156282


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [30]:
model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

del model
tf.keras.backend.clear_session()

model = tf.keras.models.load_model(current_model_path)

Epoch 1/200
73/75 [============================>.] - ETA: 0s - loss: 4.3010 - accuracy: 0.5929
Epoch 1: val_accuracy improved from -inf to 0.54612, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kmer\5fold\models\_fullModel.hdf5
75/75 [==============================] - 3s 28ms/step - loss: 4.2842 - accuracy: 0.5945 - val_loss: 3.5902 - val_accuracy: 0.5461
Epoch 2/200
72/75 [===========================>..] - ETA: 0s - loss: 3.0465 - accuracy: 0.7795
Epoch 2: val_accuracy improved from 0.54612 to 0.71347, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kmer\5fold\models\_fullModel.hdf5
75/75 [==============================] - 2s 23ms/step - loss: 3.0429 - accuracy: 0.7779 - val_loss: 2.9867 - val_accuracy: 0.7135
Epoch 3/200
74/75 [============================>.] - ETA: 0s - loss: 2.6310 - accuracy: 0.8277
Epoch 3: val_accuracy did not improve from 0.71347
75/75 [==============================] - 1s 15ms/step - loss: 2.6300 - accuracy: 0.8283 - val_loss: 2.7359 

Epoch 30/200
74/75 [============================>.] - ETA: 0s - loss: 0.5092 - accuracy: 0.9628
Epoch 30: val_accuracy did not improve from 0.71347
75/75 [==============================] - 1s 14ms/step - loss: 0.5088 - accuracy: 0.9631 - val_loss: 2.0346 - val_accuracy: 0.5535
Epoch 31/200
71/75 [===========================>..] - ETA: 0s - loss: 0.5206 - accuracy: 0.9586
Epoch 31: val_accuracy did not improve from 0.71347
75/75 [==============================] - 1s 14ms/step - loss: 0.5241 - accuracy: 0.9563 - val_loss: 1.8163 - val_accuracy: 0.5796
Epoch 32/200
72/75 [===========================>..] - ETA: 0s - loss: 0.5341 - accuracy: 0.9457
Epoch 32: val_accuracy did not improve from 0.71347
75/75 [==============================] - 1s 14ms/step - loss: 0.5330 - accuracy: 0.9458 - val_loss: 1.6719 - val_accuracy: 0.6661
Epoch 33/200
75/75 [==============================] - ETA: 0s - loss: 0.5361 - accuracy: 0.9479
Epoch 33: val_accuracy did not improve from 0.71347
75/75 [===========

74/75 [============================>.] - ETA: 0s - loss: 0.4654 - accuracy: 0.9628
Epoch 59: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 15ms/step - loss: 0.4662 - accuracy: 0.9626 - val_loss: 2.2825 - val_accuracy: 0.6016
Epoch 60/200
75/75 [==============================] - ETA: 0s - loss: 0.4657 - accuracy: 0.9631
Epoch 60: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 14ms/step - loss: 0.4657 - accuracy: 0.9631 - val_loss: 2.3133 - val_accuracy: 0.5984
Epoch 61/200
75/75 [==============================] - ETA: 0s - loss: 0.4616 - accuracy: 0.9542
Epoch 61: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 14ms/step - loss: 0.4616 - accuracy: 0.9542 - val_loss: 1.7498 - val_accuracy: 0.6245
Epoch 62/200
75/75 [==============================] - ETA: 0s - loss: 0.4725 - accuracy: 0.9639
Epoch 62: val_accuracy did not improve from 0.71755
75/75 [========================

75/75 [==============================] - 1s 14ms/step - loss: 0.4358 - accuracy: 0.9647 - val_loss: 2.1352 - val_accuracy: 0.6163
Epoch 89/200
74/75 [============================>.] - ETA: 0s - loss: 0.4140 - accuracy: 0.9649
Epoch 89: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 14ms/step - loss: 0.4140 - accuracy: 0.9652 - val_loss: 2.6731 - val_accuracy: 0.5469
Epoch 90/200
72/75 [===========================>..] - ETA: 0s - loss: 0.4029 - accuracy: 0.9718
Epoch 90: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 14ms/step - loss: 0.4037 - accuracy: 0.9719 - val_loss: 1.8710 - val_accuracy: 0.6612
Epoch 91/200
72/75 [===========================>..] - ETA: 0s - loss: 0.4517 - accuracy: 0.9627
Epoch 91: val_accuracy did not improve from 0.71755
75/75 [==============================] - 1s 14ms/step - loss: 0.4507 - accuracy: 0.9626 - val_loss: 2.7878 - val_accuracy: 0.5584
Epoch 92/200
73/75 [================

75/75 [==============================] - 1s 14ms/step - loss: 0.4411 - accuracy: 0.9635 - val_loss: 1.8513 - val_accuracy: 0.6359
Epoch 118/200
75/75 [==============================] - ETA: 0s - loss: 0.4393 - accuracy: 0.9610
Epoch 118: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4393 - accuracy: 0.9610 - val_loss: 2.2205 - val_accuracy: 0.6155
Epoch 119/200
74/75 [============================>.] - ETA: 0s - loss: 0.3955 - accuracy: 0.9688
Epoch 119: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.3952 - accuracy: 0.9689 - val_loss: 1.8063 - val_accuracy: 0.6457
Epoch 120/200
71/75 [===========================>..] - ETA: 0s - loss: 0.4224 - accuracy: 0.9643
Epoch 120: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4196 - accuracy: 0.9647 - val_loss: 2.0001 - val_accuracy: 0.6229
Epoch 121/200
71/75 [=========

Epoch 147/200
75/75 [==============================] - ETA: 0s - loss: 0.4000 - accuracy: 0.9673
Epoch 147: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4000 - accuracy: 0.9673 - val_loss: 2.2869 - val_accuracy: 0.5976
Epoch 148/200
73/75 [============================>.] - ETA: 0s - loss: 0.4316 - accuracy: 0.9628
Epoch 148: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4305 - accuracy: 0.9631 - val_loss: 2.2497 - val_accuracy: 0.5771
Epoch 149/200
71/75 [===========================>..] - ETA: 0s - loss: 0.4035 - accuracy: 0.9670
Epoch 149: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4029 - accuracy: 0.9668 - val_loss: 2.6832 - val_accuracy: 0.5551
Epoch 150/200
74/75 [============================>.] - ETA: 0s - loss: 0.3940 - accuracy: 0.9696
Epoch 150: val_accuracy did not improve from 0.72082
75/75 [===

71/75 [===========================>..] - ETA: 0s - loss: 0.4050 - accuracy: 0.9613
Epoch 176: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4082 - accuracy: 0.9610 - val_loss: 1.6802 - val_accuracy: 0.6237
Epoch 177/200
74/75 [============================>.] - ETA: 0s - loss: 0.3826 - accuracy: 0.9734
Epoch 177: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.3822 - accuracy: 0.9736 - val_loss: 2.5353 - val_accuracy: 0.6343
Epoch 178/200
74/75 [============================>.] - ETA: 0s - loss: 0.4155 - accuracy: 0.9586
Epoch 178: val_accuracy did not improve from 0.72082
75/75 [==============================] - 1s 14ms/step - loss: 0.4150 - accuracy: 0.9589 - val_loss: 1.9558 - val_accuracy: 0.6359
Epoch 179/200
74/75 [============================>.] - ETA: 0s - loss: 0.3964 - accuracy: 0.9671
Epoch 179: val_accuracy did not improve from 0.72082
75/75 [=================

In [31]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

del model
tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.720816,0.287462,0.641512,0.463054,0.772016,0.197587


In [32]:
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Independent	0.701224	0.26513	0.656614	0.453202	0.750489	0.168087

In [33]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.88      0.77      0.82      1022
           1       0.29      0.46      0.35       203

    accuracy                           0.72      1225
   macro avg       0.58      0.62      0.59      1225
weighted avg       0.78      0.72      0.74      1225



In [34]:
# 2048
# 5
# 0.001

#               precision    recall  f1-score   support

#            0       0.89      0.72      0.79      1022
#            1       0.28      0.54      0.36       203

#     accuracy                           0.69      1225
#    macro avg       0.58      0.63      0.58      1225
# weighted avg       0.79      0.69      0.72      1225

In [35]:
# 2048
# 5

#               precision    recall  f1-score   support

#            0       0.88      0.78      0.83      1022
#            1       0.29      0.45      0.35       203

#     accuracy                           0.73      1225
#    macro avg       0.58      0.62      0.59      1225
# weighted avg       0.78      0.73      0.75      1225

In [36]:
# 1024
# 5

# Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Independent	0.682449	0.259067	0.633333	0.492611	0.720157	0.170296

#               precision    recall  f1-score   support

#            0       0.88      0.72      0.79      1022
#            1       0.26      0.49      0.34       203

#     accuracy                           0.68      1225
#    macro avg       0.57      0.61      0.57      1225
# weighted avg       0.77      0.68      0.72      1225

In [37]:
# 256
# 6
# precision    recall  f1-score   support

#            0       0.88      0.68      0.77      1022
#            1       0.24      0.52      0.33       203

#     accuracy                           0.65      1225
#    macro avg       0.56      0.60      0.55      1225
# weighted avg       0.77      0.65      0.69      1225

In [38]:
# 128
# 5
#               precision    recall  f1-score   support

#            0       0.88      0.62      0.73      1022
#            1       0.23      0.59      0.34       203

#     accuracy                           0.61      1225
#    macro avg       0.56      0.60      0.53      1225
# weighted avg       0.78      0.61      0.66      1225